In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)
    
import utils
from api.gui.utils import tables
from api.gui.pairs import tab_ as pairs_tab
from api.gui.details import tab_ as details_tab
from api.gui.regression import tab_ as regression_tab
from api.gui.base import MultiTab, Tab
from api.gui.pivot import pivot_ui

In [3]:
data_path = root_path / 'data'

In [4]:
with open(data_path / 'pairs.json') as f:
    pairs = json.load(f)

In [5]:
indices_history = tables.indices_history

In [6]:
date_columns = [
    'issue_date',
    'maturity',
    'calc_maturity',
    'workout_date'
]

ref_data = pd.read_csv(data_path / 'bonds_reference.csv', index_col=['security'], parse_dates=date_columns) 
mkt_data = pd.read_csv(data_path / 'bonds_market.csv', index_col=['security'])

In [7]:
data = pd.concat([ref_data, mkt_data], axis=1, join='inner')

In [8]:
def add_years_to_maturity(df):
    df = df.assign( 
        years_to_maturity=(df['workout_date'] - pd.Timestamp.today()).dt.days / 365.2425
    )
    return df
    
    
def add_maturity_bucket(df):
    bins = [*tables.maturity['lower_bound'], np.inf]
    idx = np.digitize(df['years_to_maturity'].fillna(0), bins) - 1
    maturity_bucket = tables.maturity.iloc[idx, tables.maturity.columns.get_loc('label')].set_axis(df.index)
    maturity_bucket[df['years_to_maturity'].isna()] = np.nan
    return pd.concat([df, maturity_bucket.rename('maturity_bucket')], axis=1)

In [9]:
data = add_years_to_maturity(data)
data = add_maturity_bucket(data)

In [10]:
cols = [
    'ticker', 
    'name', 
    'currency', 
    'country', 
    'bics_sector',
    'capital_stack', 
    'maturity_bucket',
    'rating_bucket', 
    'area',
    'last_price', 
    'yield_to_call',
    'ytm_flat',
    'ytm_fwd', 
    'xccy_usd', 
    'i_spread', 
    'z_spread', 
    'aws_spread'
]

In [11]:
currency_items = data['currency'].drop_duplicates().to_list()
capital_stack_items = data['capital_stack'].drop_duplicates().to_list()
items = [
    'yield_to_call',
    'ytm_flat',
    'ytm_fwd', 
    'xccy_usd', 
    'i_spread', 
    'z_spread', 
    'aws_spread'
]

In [12]:
curr_sel = v.Autocomplete(
    v_model='EUR',
    items=currency_items,
    label='currency',
    dense=True,
    outlined=True
)

cap_stack_sel = v.Autocomplete(
    v_model='snp',
    items=capital_stack_items,
    label='capital stack',
    dense=True,
    outlined=True
)

spread_sel = v.Autocomplete(
    v_model='z_spread',
    items=items,
    label='field',
    dense=True,
    outlined=True
)


index_ac = v.Autocomplete(
    v_model=['country', 'ticker'],
    label='index',
    items=[
        'ticker',
        'country',
        'currency',
        'maturity_bucket',
        'rating_bucket',
    ],
    outlined=True,
    multiple=True,
    chips=True,
    dense=True,
    clearable=True,
    # style_='width: 250px',
    # class_='ma-0 pa-0'
)

relevant_fields = [
    'ticker',
    'country',
    'currency',
    'maturity_bucket',
    'rating_bucket',
]

cols_ac = v.Autocomplete(
    v_model=['maturity_bucket'],
    label='cols',
    items=relevant_fields,
    outlined=True,
    multiple=True,
    chips=True,
    dense=True,
    clearable=True,
    # style_='width: 250px',
    # class_='ma-0 pa-0'
)

table_btn = v.Btn(
    left=True,
    children=[
        v.Icon(children=['mdi-table']),
        'Table',
    ],
)

out = w.Output()


param_box = v.Container(
    children=[
        v.Row(
            children=[
                v.Col(
                    cols=2,
                    children=[curr_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[cap_stack_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[spread_sel],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=2,
                    children=[table_btn],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        ),
        v.Row(
            children=[
                v.Col(
                    cols=3,
                    children=[index_ac],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=3,
                    children=[cols_ac],
                    class_="my-0 py-0"
                ),
            ],
            align_content='center',
        )
    ]
)

tab = w.VBox(
    children=[
        param_box,
        out,
    ]
)


In [13]:
def get_col():
    return [
        *relevant_fields,
        spread_sel.v_model,
    ]

def get_mat():
    return [
        '2yr',
        '5yr',
        '7yr',
        '10yr',
        '15yr'
    ]


def get_rtg():
    return [
        'AA',
        'A',
        'BBB',
        'BB',
        'B'
    ]

In [14]:
def get_pivot(df):
    filters = [
        f"currency == '{curr_sel.v_model}'",
        " and ", f"capital_stack == '{cap_stack_sel.v_model}'",
        " and ", f"maturity_bucket in {get_mat()}",
        " and ", f"rating_bucket in {get_rtg()}"
    ]
    
    df_ = df.query(''.join(filters))
    return df_.dropna(subset=spread_sel.v_model).pivot_table(values=spread_sel.v_model, index=index_ac.v_model, columns=cols_ac.v_model)

In [15]:
def make_data_table(df, **kwargs):
    dfj = df_to_json(df)
    t =  v.DataTable(
        v_model=None,
        headers=dfj['headers'],
        items=dfj['display_items'],
        multi_sort=True,
        class_="elevation-1",
        dense=True,
        items_per_page=100,
    )
    return t

def make_single_item_container(item):
    return v.Container(
        children=[

            v.Row(
                children=[
                    v.Col(
                        cols=12,
                        children=[
                            item
                        ]
                    )
                ]
            )
        ]
    )


In [16]:
def custom_sort(df, axis=1):
    idx = df.columns if axis == 1 else df.index
    
    spec = {
        'maturity_bucket': lambda x: x.map(tables.maturity.set_index('label')['rank']), 
        'rating_bucket': lambda x: x.map(tables.rating.set_index('rating')['rank'])
    }
    
    for col in idx.names:
        key = spec.get(col)
        if key:
             df = df.sort_index(level=col, key=key, axis=axis)
    return df

In [17]:
df = data
def on_click_make_pivot(widgets, event, data):
    pivot_df = get_pivot(df)
    pivot_df = custom_sort(pivot_df, axis=1)
    pivot_df = custom_sort(pivot_df, axis=0)
    data_table = make_data_table(pivot_df)
    cont = make_single_item_container(data_table)
    with out:
        out.clear_output()
        display(cont)
    
table_btn.on_event('click', on_click_make_pivot)

In [18]:
p = get_pivot(data)

In [20]:
from ipytables import ITable

In [22]:
p

maturity_bucket        10yr        2yr         5yr         7yr
country ticker                                                
AT      BAWAG           NaN        NaN  100.190673         NaN
        OBERBK          NaN        NaN         NaN   97.567707
BE      ARGSPA          NaN        NaN  135.086182         NaN
        CCBGBB          NaN  47.775928   94.320130         NaN
        KBCBB    104.401432  26.021599  108.676644   96.375029
...                     ...        ...         ...         ...
US      C               NaN  35.020546         NaN  100.199627
        GS       128.167415  57.768137         NaN  121.540345
        JPM       93.595754  21.462570  101.557997   82.530847
        MS       118.698653  39.337133  102.220039  113.810081
        WFC             NaN  65.226926  110.184528  124.829924

[62 rows x 4 columns]

In [21]:
ITable(df=p)

ITable(value=None, data={'data': '[{"country":"AT","ticker":"BAWAG","10yr":null,"2yr":null,"5yr":100.1907,"7yr…

In [ ]:
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader

In [ ]:
template_path = root_path / 'api/gui'

In [ ]:
env = Environment(
    loader=FileSystemLoader(template_path),
    autoescape=False
)

In [ ]:
tt = env.get_template('table_template.html')
# tt = env.get_template('test.html')

In [ ]:
from html import escape
from api.gui.pivot import IFrame

In [ ]:
dta = p.reset_index().to_json(
    orient='records',
    double_precision=4,
    index=True
)
cols = json.dumps([{"data": col, "title": col } for col in p.reset_index()])

In [ ]:
render = tt.render(data=dta, cols=cols)

In [ ]:
render

In [ ]:
IFrame(escape(render), height=500)

In [ ]:
from itables import init_notebook_mode

In [ ]:
init_notebook_mode(all_interactive=True)

In [ ]:
p

In [ ]:
from IPython.display import HTML

HTML(tt.render(data=dta, cols=cols))

In [ ]:
p.sort_inde